In [2]:
import os
import numpy as np
import pandas as pd
import shutil
import cv2
from sklearn.model_selection import GroupShuffleSplit
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
# Diretórios
train_dir = 'dataset/processado/train/'
val_dir = 'dataset/processado/validation/'
test_dir = 'dataset/processado/test/'

In [4]:
def extract_patient_id(filename):
    """Extrai o ID do paciente do nome do arquivo"""
    basename = os.path.basename(filename)
    # Formato esperado: UID_X_Y_Z_all.bmp ou semelhante
    try:
        patient_id = basename.split('_')[1].split('-')[0]
        return patient_id
    except:
        # Fallback para outros formatos
        return basename.split('_')[0]

In [5]:
def collect_images_and_patients(directory):
    """Coleta todas as imagens e seus respectivos IDs de pacientes"""
    images = []
    patients = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.jpg', '.png', '.bmp')):
                img_path = os.path.join(root, file)
                patient_id = extract_patient_id(file)
                
                images.append(img_path)
                patients.append(patient_id)
    
    return np.array(images), np.array(patients)

In [6]:
def split_by_proportion(unique_ids, proportion, random_state):
    """Divide uma lista de IDs únicos pela proporção dada"""
    np.random.seed(random_state)
    np.random.shuffle(unique_ids)
    
    split_idx = int(len(unique_ids) * proportion)
    return unique_ids[:split_idx], unique_ids[split_idx:]

In [7]:
def copy_images(image_paths, target_dir):
    """Copia as imagens para o diretório de destino"""
    os.makedirs(target_dir, exist_ok=True)
    
    for img_path in tqdm(image_paths, desc=f"Copiando para {os.path.basename(target_dir)}"):
        shutil.copy2(img_path, os.path.join(target_dir, os.path.basename(img_path)))


In [8]:
def calculate_augmentation_per_patient(patient_ids, images, all_patients, total_needed):
    """Calcula quantas imagens augmentadas são necessárias por paciente"""
    # Mapear pacientes para imagens
    patient_to_images = {}
    for img, patient in zip(images, all_patients):
        if patient in patient_ids:
            if patient not in patient_to_images:
                patient_to_images[patient] = []
            patient_to_images[patient].append(img)
    
    # Calcular a proporção de imagens por paciente
    total_images = len(images)
    augmentation_per_patient = {}
    
    for patient in patient_ids:
        patient_images = patient_to_images.get(patient, [])
        # Calcular a proporção do paciente
        proportion = len(patient_images) / total_images if total_images > 0 else 0
        # Calcular o número de imagens augmentadas
        aug_count = max(5, int(proportion * total_needed))
        augmentation_per_patient[patient] = aug_count
    
    # Ajustar para garantir o total correto
    total_augmentation = sum(augmentation_per_patient.values())
    if total_augmentation < total_needed:
        # Distribuir a diferença entre os pacientes
        difference = total_needed - total_augmentation
        # Adicionar a diferença aos pacientes com mais imagens originais
        sorted_patients = sorted(patient_ids, 
                                key=lambda p: len(patient_to_images.get(p, [])), 
                                reverse=True)
        
        for i, patient in enumerate(sorted_patients):
            if i >= difference:
                break
            augmentation_per_patient[patient] += 1
    
    return augmentation_per_patient

In [9]:
def apply_augmentation_by_patient(images, all_patients, target_patients, aug_per_patient, output_dir, aggressive=False):
    """Aplica data augmentation por paciente"""
    # Agrupar imagens por paciente
    patient_to_images = {}
    for img, patient in zip(images, all_patients):
        if patient in target_patients:
            if patient not in patient_to_images:
                patient_to_images[patient] = []
            patient_to_images[patient].append(img)
    
    # Aplicar augmentation para cada paciente
    for patient, imgs in tqdm(patient_to_images.items(), desc="Augmentação por paciente"):
        target_aug_count = aug_per_patient.get(patient, 0)
        if target_aug_count <= 0:
            continue
        
        # Distribuir augmentações entre as imagens originais
        imgs_needed_per_original = max(1, target_aug_count // len(imgs))
        
        aug_count = 0
        aug_round = 0
        
        # Continuar aplicando rounds de augmentação até atingir o número necessário
        while aug_count < target_aug_count:
            aug_round += 1
            for img_path in imgs:
                # Ler imagem
                img = cv2.imread(img_path)
                if img is None:
                    continue
                
                basename = os.path.basename(img_path)
                name_without_ext = os.path.splitext(basename)[0]
                
                # Determinar o número de augmentações para esta imagem neste round
                n_augs = min(imgs_needed_per_original, target_aug_count - aug_count)
                if n_augs <= 0:
                    break
                
                # Aplicar mais transformações se aggressive=True
                n_transforms = random.randint(3, 5) if aggressive else random.randint(1, 3)
                
                # Gerar augmentações
                for i in range(n_augs):
                    # Aplicar transformações aleatórias
                    aug_img = apply_random_augmentation(img, n_transforms=n_transforms)
                    
                    # Salvar imagem augmentada
                    aug_filename = f"{name_without_ext}_aug_r{aug_round}_{i}.png"
                    cv2.imwrite(os.path.join(output_dir, aug_filename), aug_img)
                    
                    aug_count += 1
                    if aug_count >= target_aug_count:
                        break
                
                if aug_count >= target_aug_count:
                    break

In [10]:
def apply_random_augmentation(image, n_transforms=2):
    """Aplica transformações aleatórias na imagem"""
    # Lista de possíveis augmentações
    augmentations = [
        lambda img: cv2.flip(img, 1),  # Espelhamento horizontal
        lambda img: cv2.flip(img, 0),  # Espelhamento vertical
        lambda img: cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE),  # Rotação 90°
        lambda img: cv2.rotate(img, cv2.ROTATE_180),  # Rotação 180°
        lambda img: cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE),  # Rotação 270°
        lambda img: cv2.GaussianBlur(img, (5, 5), 0),  # Blur gaussiano
        lambda img: add_salt_pepper_noise(img, 0.02),  # Ruído sal e pimenta
        lambda img: apply_shear(img, random.uniform(0.1, 0.3)),  # Cisalhamento
        lambda img: adjust_brightness(img, factor=random.uniform(0.8, 1.2)),  # Ajuste de brilho
        lambda img: adjust_contrast(img, factor=random.uniform(0.8, 1.2)),  # Ajuste de contraste
        lambda img: apply_random_crop(img, crop_percent=random.uniform(0.8, 0.95)),  # Corte aleatório
        lambda img: apply_elastic_transform(img, alpha=random.randint(40, 60), sigma=random.randint(4, 6))  # Transformação elástica
    ]
    
    # Escolher quantas augmentações aplicar
    num_augs = min(n_transforms, len(augmentations))
    
    # Escolher augmentações aleatórias
    selected_augs = random.sample(augmentations, num_augs)
    
    # Aplicar augmentações
    result = image.copy()
    for aug_func in selected_augs:
        result = aug_func(result)
    
    return result

In [11]:
def add_salt_pepper_noise(image, amount):
    """Adiciona ruído sal e pimenta na imagem"""
    output = image.copy()
    h, w = image.shape[:2]
    num_salt = int(amount * image.size * 0.5)
    num_pepper = int(amount * image.size * 0.5)
    
    # Salt
    coords = [np.random.randint(0, i - 1, num_salt) for i in (h, w)]
    output[coords[0], coords[1]] = 255
    
    # Pepper
    coords = [np.random.randint(0, i - 1, num_pepper) for i in (h, w)]
    output[coords[0], coords[1]] = 0
    
    return output

In [12]:
def apply_shear(image, factor):
    """Aplica transformação de cisalhamento na imagem"""
    h, w = image.shape[:2]
    
    # Matriz de transformação
    M = np.float32([[1, factor, 0], [0, 1, 0]])
    
    # Aplicar transformação
    sheared = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REPLICATE)
    
    return sheared

In [13]:
def adjust_brightness(image, factor):
    """Ajusta o brilho da imagem"""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = hsv.astype(np.float32)
    
    # Ajustar o canal V (brilho)
    hsv[:, :, 2] = hsv[:, :, 2] * factor
    hsv[:, :, 2] = np.clip(hsv[:, :, 2], 0, 255)
    
    hsv = hsv.astype(np.uint8)
    result = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    return result

In [14]:
def adjust_contrast(image, factor):
    """Ajusta o contraste da imagem"""
    mean = np.mean(image, axis=(0, 1))
    result = (image.astype(np.float32) - mean) * factor + mean
    result = np.clip(result, 0, 255).astype(np.uint8)
    return result


In [15]:
def apply_random_crop(image, crop_percent=0.9):
    """Aplica corte aleatório e redimensiona para o tamanho original"""
    h, w = image.shape[:2]
    
    # Calcular dimensões do corte
    crop_h = int(h * crop_percent)
    crop_w = int(w * crop_percent)
    
    # Calcular posição do corte
    start_h = random.randint(0, h - crop_h)
    start_w = random.randint(0, w - crop_w)
    
    # Cortar imagem
    cropped = image[start_h:start_h+crop_h, start_w:start_w+crop_w]
    
    # Redimensionar para o tamanho original
    resized = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)
    
    return resized

In [16]:
def apply_elastic_transform(image, alpha=50, sigma=5, random_state=None):
    """Aplica transformação elástica à imagem"""
    if random_state is None:
        random_state = np.random.RandomState(None)
    
    shape = image.shape
    dx = random_state.rand(shape[0], shape[1]) * 2 - 1
    dy = random_state.rand(shape[0], shape[1]) * 2 - 1
    dx = cv2.GaussianBlur(dx, (0, 0), sigma) * alpha
    dy = cv2.GaussianBlur(dy, (0, 0), sigma) * alpha
    
    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    map_x = np.float32(x + dx)
    map_y = np.float32(y + dy)
    
    return cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)


In [17]:
def count_images_in_directory(train_dir, val_dir, test_dir):
    """Conta o número de imagens em cada diretório"""
    # Treino
    all_train_count = len([f for f in os.listdir(os.path.join(train_dir, 'all')) 
                           if f.endswith(('.jpg', '.png', '.bmp'))])
    hem_train_count = len([f for f in os.listdir(os.path.join(train_dir, 'hem')) 
                           if f.endswith(('.jpg', '.png', '.bmp'))])
    
    # Validação
    all_val_count = len([f for f in os.listdir(os.path.join(val_dir, 'all')) 
                         if f.endswith(('.jpg', '.png', '.bmp'))])
    hem_val_count = len([f for f in os.listdir(os.path.join(val_dir, 'hem')) 
                         if f.endswith(('.jpg', '.png', '.bmp'))])
    
    # Teste
    all_test_count = len([f for f in os.listdir(os.path.join(test_dir, 'all')) 
                          if f.endswith(('.jpg', '.png', '.bmp'))])
    hem_test_count = len([f for f in os.listdir(os.path.join(test_dir, 'hem')) 
                          if f.endswith(('.jpg', '.png', '.bmp'))])
    
    print("\nContagem final de imagens:")
    print(f"Treino - ALL: {all_train_count}, HEM: {hem_train_count}, Total: {all_train_count + hem_train_count}")
    print(f"Validação - ALL: {all_val_count}, HEM: {hem_val_count}, Total: {all_val_count + hem_val_count}")
    print(f"Teste - ALL: {all_test_count}, HEM: {hem_test_count}, Total: {all_test_count + hem_test_count}")


In [18]:
def plot_distribution(train_dir, val_dir, test_dir):
    """Gera gráficos para visualizar a distribuição de imagens por paciente"""
    # Criar figura para distribuição por paciente
    plt.figure(figsize=(18, 12))
    
    # Treino - ALL
    train_all_files = [f for f in os.listdir(os.path.join(train_dir, 'all')) 
                       if f.endswith(('.jpg', '.png', '.bmp'))]
    train_all_patients = [extract_patient_id(f) for f in train_all_files]
    train_all_count = Counter(train_all_patients)
    
    # Treino - HEM
    train_hem_files = [f for f in os.listdir(os.path.join(train_dir, 'hem')) 
                       if f.endswith(('.jpg', '.png', '.bmp'))]
    train_hem_patients = [extract_patient_id(f) for f in train_hem_files]
    train_hem_count = Counter(train_hem_patients)
    
    # Validação - ALL
    val_all_files = [f for f in os.listdir(os.path.join(val_dir, 'all')) 
                     if f.endswith(('.jpg', '.png', '.bmp'))]
    val_all_patients = [extract_patient_id(f) for f in val_all_files]
    val_all_count = Counter(val_all_patients)
    
    # Validação - HEM
    val_hem_files = [f for f in os.listdir(os.path.join(val_dir, 'hem')) 
                     if f.endswith(('.jpg', '.png', '.bmp'))]
    val_hem_patients = [extract_patient_id(f) for f in val_hem_files]
    val_hem_count = Counter(val_hem_patients)
    
    # Teste - ALL
    test_all_files = [f for f in os.listdir(os.path.join(test_dir, 'all')) 
                      if f.endswith(('.jpg', '.png', '.bmp'))]
    test_all_patients = [extract_patient_id(f) for f in test_all_files]
    test_all_count = Counter(test_all_patients)
    
    # Teste - HEM
    test_hem_files = [f for f in os.listdir(os.path.join(test_dir, 'hem')) 
                      if f.endswith(('.jpg', '.png', '.bmp'))]
    test_hem_patients = [extract_patient_id(f) for f in test_hem_files]
    test_hem_count = Counter(test_hem_patients)
    
    # Plotar distribuição por paciente
    plt.subplot(3, 2, 1)
    plt.bar(train_all_count.keys(), train_all_count.values())
    plt.title('Treino - ALL: Imagens por Paciente')
    plt.xlabel('ID do Paciente')
    plt.ylabel('Número de Imagens')
    plt.xticks(rotation=90)
    
    plt.subplot(3, 2, 2)
    plt.bar(train_hem_count.keys(), train_hem_count.values())
    plt.title('Treino - HEM: Imagens por Paciente')
    plt.xlabel('ID do Paciente')
    plt.ylabel('Número de Imagens')
    plt.xticks(rotation=90)
    
    plt.subplot(3, 2, 3)
    plt.bar(val_all_count.keys(), val_all_count.values())
    plt.title('Validação - ALL: Imagens por Paciente')
    plt.xlabel('ID do Paciente')
    plt.ylabel('Número de Imagens')
    plt.xticks(rotation=90)
    
    plt.subplot(3, 2, 4)
    plt.bar(val_hem_count.keys(), val_hem_count.values())
    plt.title('Validação - HEM: Imagens por Paciente')
    plt.xlabel('ID do Paciente')
    plt.ylabel('Número de Imagens')
    plt.xticks(rotation=90)
    
    plt.subplot(3, 2, 5)
    plt.bar(test_all_count.keys(), test_all_count.values())
    plt.title('Teste - ALL: Imagens por Paciente')
    plt.xlabel('ID do Paciente')
    plt.ylabel('Número de Imagens')
    plt.xticks(rotation=90)
    
    plt.subplot(3, 2, 6)
    plt.bar(test_hem_count.keys(), test_hem_count.values())
    plt.title('Teste - HEM: Imagens por Paciente')
    plt.xlabel('ID do Paciente')
    plt.ylabel('Número de Imagens')
    plt.xticks(rotation=90)
    
    plt.tight_layout()
    plt.savefig('distribuicao_imagens_por_paciente.png')
    plt.close()
    
    # Criar figura para resumo de distribuição de classes
    plt.figure(figsize=(15, 10))
    
    # Totais por conjunto
    train_counts = [len(train_all_files), len(train_hem_files)]
    val_counts = [len(val_all_files), len(val_hem_files)]
    test_counts = [len(test_all_files), len(test_hem_files)]
    
    plt.subplot(2, 2, 1)
    plt.bar(['ALL', 'HEM'], [sum(train_counts), sum(val_counts) + sum(test_counts)])
    plt.title('Distribuição de Classes (Treino vs. Validação+Teste)')
    plt.ylabel('Número de Imagens')
    
    plt.subplot(2, 2, 2)
    plt.bar(['Treino', 'Validação', 'Teste'], 
            [sum(train_counts), sum(val_counts), sum(test_counts)])
    plt.title('Distribuição por Conjunto (Total)')
    plt.ylabel('Número de Imagens')
    
    plt.subplot(2, 2, 3)
    width = 0.35
    x = np.arange(3)
    plt.bar(x - width/2, [train_counts[0], val_counts[0], test_counts[0]], width, label='ALL')
    plt.bar(x + width/2, [train_counts[1], val_counts[1], test_counts[1]], width, label='HEM')
    plt.xticks(x, ['Treino', 'Validação', 'Teste'])
    plt.title('Distribuição por Conjunto e Classe')
    plt.ylabel('Número de Imagens')
    plt.legend()
    
    plt.subplot(2, 2, 4)
    plt.pie([sum(train_counts), sum(val_counts), sum(test_counts)], 
            labels=['Treino', 'Validação', 'Teste'],
            autopct='%1.1f%%')
    plt.title('Proporção dos Conjuntos')
    
    plt.tight_layout()
    plt.savefig('resumo_distribuicao_dataset.png')
    plt.close()

In [19]:
def create_dataset_preparation_pipeline(
    all_dir,            # Diretório com imagens ALL (câncer)
    hem_dir,            # Diretório com imagens HEM (saudáveis)
    output_dir,         # Diretório de saída
    train_size=0.7,     # Proporção para treinamento
    val_size=0.2,       # Proporção para validação
    test_size=0.1,      # Proporção para teste
    target_train_per_class=10000,  # Alvo para cada classe após augmentação no treino
    target_val_per_class=2000,     # Alvo para cada classe após augmentação na validação
    target_test_per_class=1000,    # Alvo para cada classe após augmentação no teste
    random_state=42     # Semente aleatória para reprodutibilidade
):
    """
    Pipeline completo para preparação do dataset:
    1. Separação por paciente
    2. Balanceamento das classes
    3. Data augmentation
    """
    # Criar diretórios de saída
    os.makedirs(output_dir, exist_ok=True)
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'validation')
    test_dir = os.path.join(output_dir, 'test')
    
    for d in [train_dir, val_dir, test_dir]:
        os.makedirs(os.path.join(d, 'all'), exist_ok=True)
        os.makedirs(os.path.join(d, 'hem'), exist_ok=True)
    
    # 1. Coletar informações sobre imagens e pacientes
    print("Coletando informações sobre imagens e pacientes...")
    all_images, all_patients = collect_images_and_patients(all_dir)
    hem_images, hem_patients = collect_images_and_patients(hem_dir)
    
    print(f"ALL: {len(all_images)} imagens de {len(np.unique(all_patients))} pacientes")
    print(f"HEM: {len(hem_images)} imagens de {len(np.unique(hem_patients))} pacientes")
    
    # 2. Dividir pacientes em conjuntos de treino, validação e teste
    print("\nDividindo pacientes em conjuntos...")
    
    # ALL - Divisão treino/validação+teste
    all_train_patients, all_temp_patients = split_by_proportion(
        np.unique(all_patients), train_size, random_state)
    
    # ALL - Divisão validação/teste
    val_proportion = val_size / (val_size + test_size)
    all_val_patients, all_test_patients = split_by_proportion(
        all_temp_patients, val_proportion, random_state)
    
    # HEM - Divisão treino/validação+teste
    hem_train_patients, hem_temp_patients = split_by_proportion(
        np.unique(hem_patients), train_size, random_state)
    
    # HEM - Divisão validação/teste
    hem_val_patients, hem_test_patients = split_by_proportion(
        hem_temp_patients, val_proportion, random_state)
    
    print(f"ALL - Treino: {len(all_train_patients)} pacientes, Validação: {len(all_val_patients)} pacientes, Teste: {len(all_test_patients)} pacientes")
    print(f"HEM - Treino: {len(hem_train_patients)} pacientes, Validação: {len(hem_val_patients)} pacientes, Teste: {len(hem_test_patients)} pacientes")
    
    # 3. Distribuir imagens originais com base na divisão de pacientes
    print("\nDistribuindo imagens originais...")
    
    # ALL
    all_train_imgs = [img for img, patient in zip(all_images, all_patients) if patient in all_train_patients]
    all_val_imgs = [img for img, patient in zip(all_images, all_patients) if patient in all_val_patients]
    all_test_imgs = [img for img, patient in zip(all_images, all_patients) if patient in all_test_patients]
    
    # HEM
    hem_train_imgs = [img for img, patient in zip(hem_images, hem_patients) if patient in hem_train_patients]
    hem_val_imgs = [img for img, patient in zip(hem_images, hem_patients) if patient in hem_val_patients]
    hem_test_imgs = [img for img, patient in zip(hem_images, hem_patients) if patient in hem_test_patients]
    
    print(f"ALL - Treino: {len(all_train_imgs)} imagens, Validação: {len(all_val_imgs)} imagens, Teste: {len(all_test_imgs)} imagens")
    print(f"HEM - Treino: {len(hem_train_imgs)} imagens, Validação: {len(hem_val_imgs)} imagens, Teste: {len(hem_test_imgs)} imagens")
    
    # 4. Copiar imagens originais para os diretórios correspondentes
    print("\nCopiando imagens originais para seus diretórios...")
    
    # ALL
    copy_images(all_train_imgs, os.path.join(train_dir, 'all'))
    copy_images(all_val_imgs, os.path.join(val_dir, 'all'))
    copy_images(all_test_imgs, os.path.join(test_dir, 'all'))
    
    # HEM
    copy_images(hem_train_imgs, os.path.join(train_dir, 'hem'))
    copy_images(hem_val_imgs, os.path.join(val_dir, 'hem'))
    copy_images(hem_test_imgs, os.path.join(test_dir, 'hem'))
    
    # 5. Aplicar data augmentation nas imagens de treinamento
    print("\nAplicando data augmentation para balancear o conjunto de treinamento...")
    
    # Calcular quantas imagens augmentadas são necessárias para cada classe
    all_needed = max(0, target_train_per_class - len(all_train_imgs))
    hem_needed = max(0, target_train_per_class - len(hem_train_imgs))
    
    print(f"ALL: Gerando {all_needed} imagens adicionais para treinamento")
    print(f"HEM: Gerando {hem_needed} imagens adicionais para treinamento")
    
    if all_needed > 0:
        all_per_patient = calculate_augmentation_per_patient(all_train_patients, all_train_imgs, all_patients, all_needed)
        apply_augmentation_by_patient(all_train_imgs, all_patients, all_train_patients, 
                                      all_per_patient, os.path.join(train_dir, 'all'), 
                                      aggressive=True)
    
    if hem_needed > 0:
        hem_per_patient = calculate_augmentation_per_patient(hem_train_patients, hem_train_imgs, hem_patients, hem_needed)
        apply_augmentation_by_patient(hem_train_imgs, hem_patients, hem_train_patients, 
                                      hem_per_patient, os.path.join(train_dir, 'hem'),
                                      aggressive=True)
    
    # 6. Aplicar data augmentation nas imagens de validação
    print("\nAplicando data augmentation para o conjunto de validação...")
    
    # Calcular quantas imagens augmentadas são necessárias para validação
    all_val_needed = max(0, target_val_per_class - len(all_val_imgs))
    hem_val_needed = max(0, target_val_per_class - len(hem_val_imgs))
    
    print(f"ALL: Gerando {all_val_needed} imagens adicionais para validação")
    print(f"HEM: Gerando {hem_val_needed} imagens adicionais para validação")
    
    if all_val_needed > 0:
        all_val_per_patient = calculate_augmentation_per_patient(all_val_patients, all_val_imgs, all_patients, all_val_needed)
        apply_augmentation_by_patient(all_val_imgs, all_patients, all_val_patients, 
                                      all_val_per_patient, os.path.join(val_dir, 'all'),
                                      aggressive=True)
    
    if hem_val_needed > 0:
        hem_val_per_patient = calculate_augmentation_per_patient(hem_val_patients, hem_val_imgs, hem_patients, hem_val_needed)
        apply_augmentation_by_patient(hem_val_imgs, hem_patients, hem_val_patients, 
                                      hem_val_per_patient, os.path.join(val_dir, 'hem'),
                                      aggressive=True)
    
    # 7. Aplicar data augmentation nas imagens de teste
    print("\nAplicando data augmentation para o conjunto de teste...")
    
    # Calcular quantas imagens augmentadas são necessárias para teste
    all_test_needed = max(0, target_test_per_class - len(all_test_imgs))
    hem_test_needed = max(0, target_test_per_class - len(hem_test_imgs))
    
    print(f"ALL: Gerando {all_test_needed} imagens adicionais para teste")
    print(f"HEM: Gerando {hem_test_needed} imagens adicionais para teste")
    
    if all_test_needed > 0:
        all_test_per_patient = calculate_augmentation_per_patient(all_test_patients, all_test_imgs, all_patients, all_test_needed)
        apply_augmentation_by_patient(all_test_imgs, all_patients, all_test_patients, 
                                     all_test_per_patient, os.path.join(test_dir, 'all'),
                                     aggressive=True)
    
    if hem_test_needed > 0:
        hem_test_per_patient = calculate_augmentation_per_patient(hem_test_patients, hem_test_imgs, hem_patients, hem_test_needed)
        apply_augmentation_by_patient(hem_test_imgs, hem_patients, hem_test_patients, 
                                     hem_test_per_patient, os.path.join(test_dir, 'hem'),
                                     aggressive=True)
    
    # 8. Verificar resultados finais
    print("\nVerificando números finais...")
    count_images_in_directory(train_dir, val_dir, test_dir)
    
    # 9. Gerar visualização das distribuições de imagens por paciente
    print("\nGerando visualização das distribuições...")
    plot_distribution(train_dir, val_dir, test_dir)
    
    # Salvar um arquivo de texto com a distribuição do dataset
    with open('distribuicao_dataset.txt', 'w') as f:
        f.write(f"ALL: {len(all_images)} imagens de {len(np.unique(all_patients))} pacientes\n")
        f.write(f"HEM: {len(hem_images)} imagens de {len(np.unique(hem_patients))} pacientes\n\n")
        
        f.write("Dividindo pacientes em conjuntos...\n")
        f.write(f"ALL - Treino: {len(all_train_patients)} pacientes, Validação: {len(all_val_patients)} pacientes, Teste: {len(all_test_patients)} pacientes\n")
        f.write(f"HEM - Treino: {len(hem_train_patients)} pacientes, Validação: {len(hem_val_patients)} pacientes, Teste: {len(hem_test_patients)} pacientes\n\n")
        
        f.write("Distribuindo imagens originais...\n")
        f.write(f"ALL - Treino: {len(all_train_imgs)} imagens, Validação: {len(all_val_imgs)} imagens, Teste: {len(all_test_imgs)} imagens\n")
        f.write(f"HEM - Treino: {len(hem_train_imgs)} imagens, Validação: {len(hem_val_imgs)} imagens, Teste: {len(hem_test_imgs)} imagens\n\n")
        
        # Incluir contagem final após augmentação
        train_all_count = len([f for f in os.listdir(os.path.join(train_dir, 'all')) 
                              if f.endswith(('.jpg', '.png', '.bmp'))])
        train_hem_count = len([f for f in os.listdir(os.path.join(train_dir, 'hem')) 
                              if f.endswith(('.jpg', '.png', '.bmp'))])
        
        val_all_count = len([f for f in os.listdir(os.path.join(val_dir, 'all')) 
                            if f.endswith(('.jpg', '.png', '.bmp'))])
        val_hem_count = len([f for f in os.listdir(os.path.join(val_dir, 'hem')) 
                            if f.endswith(('.jpg', '.png', '.bmp'))])
        
        test_all_count = len([f for f in os.listdir(os.path.join(test_dir, 'all')) 
                             if f.endswith(('.jpg', '.png', '.bmp'))])
        test_hem_count = len([f for f in os.listdir(os.path.join(test_dir, 'hem')) 
                             if f.endswith(('.jpg', '.png', '.bmp'))])
        
        f.write("Contagem final de imagens:\n")
        f.write(f"Treino - ALL: {train_all_count}, HEM: {train_hem_count}, Total: {train_all_count + train_hem_count}\n")
        f.write(f"Validação - ALL: {val_all_count}, HEM: {val_hem_count}, Total: {val_all_count + val_hem_count}\n")
        f.write(f"Teste - ALL: {test_all_count}, HEM: {test_hem_count}, Total: {test_all_count + test_hem_count}\n\n")
        
        f.write("- Treino: dataset/processado/train\n")
        f.write("- Validação: dataset/processado/validation\n")
        f.write("- Teste: dataset/processado/test")
    
    print("\nPreparação do dataset concluída com sucesso!")
    
    return train_dir, val_dir, test_dir

# Exemplo de uso
if __name__ == "__main__":
    all_dir = "dataset/all"  # Diretório com imagens ALL (câncer)
    hem_dir = "dataset/hem"  # Diretório com imagens HEM (saudáveis)
    output_dir = "dataset/processado"  # Diretório de saída
    
    train_dir, val_dir, test_dir = create_dataset_preparation_pipeline(
        all_dir=all_dir,
        hem_dir=hem_dir,
        output_dir=output_dir,
        target_train_per_class=10000,  # 10.000 imagens por classe para treinamento
        target_val_per_class=2000,     # 2.000 imagens por classe para validação
        target_test_per_class=1000      # 1.000 imagens por classe para teste
    )
    
    print(f"Dataset preparado com sucesso:")
    print(f"- Treino: {train_dir}")
    print(f"- Validação: {val_dir}")
    print(f"- Teste: {test_dir}")

Coletando informações sobre imagens e pacientes...
ALL: 7272 imagens de 47 pacientes
HEM: 3389 imagens de 26 pacientes

Dividindo pacientes em conjuntos...
ALL - Treino: 32 pacientes, Validação: 10 pacientes, Teste: 5 pacientes
HEM - Treino: 18 pacientes, Validação: 5 pacientes, Teste: 3 pacientes

Distribuindo imagens originais...
ALL - Treino: 4836 imagens, Validação: 1716 imagens, Teste: 720 imagens
HEM - Treino: 2580 imagens, Validação: 582 imagens, Teste: 227 imagens

Copiando imagens originais para seus diretórios...


Copiando para hem: 100%|██████████| 227/227 [00:08<00:00, 27.44it/s]



Aplicando data augmentation para balancear o conjunto de treinamento...
ALL: Gerando 5164 imagens adicionais para treinamento
HEM: Gerando 7420 imagens adicionais para treinamento


Augmentação por paciente: 100%|██████████| 11/11 [02:24<00:00, 13.13s/it]



Aplicando data augmentation para o conjunto de validação...
ALL: Gerando 284 imagens adicionais para validação
HEM: Gerando 1418 imagens adicionais para validação


Augmentação por paciente: 100%|██████████| 2/2 [00:00<00:00,  2.17it/s]
Augmentação por paciente: 0it [00:00, ?it/s]



Aplicando data augmentation para o conjunto de teste...
ALL: Gerando 280 imagens adicionais para teste
HEM: Gerando 773 imagens adicionais para teste


Augmentação por paciente: 100%|██████████| 1/1 [00:01<00:00,  1.17s/it]
Augmentação por paciente: 0it [00:00, ?it/s]



Verificando números finais...

Contagem final de imagens:
Treino - ALL: 8292, HEM: 8092, Total: 16384
Validação - ALL: 1744, HEM: 582, Total: 2326
Teste - ALL: 749, HEM: 227, Total: 976

Gerando visualização das distribuições...

Preparação do dataset concluída com sucesso!
Dataset preparado com sucesso:
- Treino: dataset/processado/train
- Validação: dataset/processado/validation
- Teste: dataset/processado/test
